In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.utils.class_weight import compute_class_weight
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Conv1D, MaxPooling1D, LSTM, Dense, Dropout, BatchNormalization
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# ==================== LOAD PRE-SPLIT DATA (No Data Leakage) ====================
print("="*70)
print("LOADING PRE-SPLIT DATA (No Data Leakage)")
print("="*70)

X_train = np.load(r"preprocessed\TRAIN_X.npy")
y_train = np.load(r"preprocessed\TRAIN_y.npy")
X_test = np.load(r"preprocessed\TEST_X.npy")
y_test = np.load(r"preprocessed\TEST_y.npy")

# Convert to binary: ICTAL = 0, ALL OTHERS = 1
y_train_encoded = np.where(y_train == 'ICTAL', 0, 1)
y_test_encoded = np.where(y_test == 'ICTAL', 0, 1)

# Split training data to create validation set
X_train_final, X_val, y_train_final, y_val = train_test_split(
    X_train, y_train_encoded, test_size=0.1765, stratify=y_train_encoded, random_state=42
)

# Reshape for CNN input
X_train_final = X_train_final.reshape((X_train_final.shape[0], X_train_final.shape[1], 1))
X_val = X_val.reshape((X_val.shape[0], X_val.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

print(f"\nDataset shapes:")
print(f"  Train:      {X_train_final.shape}")
print(f"  Validation: {X_val.shape}")
print(f"  Test:       {X_test.shape}")

print(f"\nClass distribution:")
print(f"  Train: {np.unique(y_train_final, return_counts=True)}")
print(f"  Val:   {np.unique(y_val, return_counts=True)}")
print(f"  Test:  {np.unique(y_test_encoded, return_counts=True)}")

# ==================== HYBRID FOCAL LOSS ====================
def hybrid_focal_loss(alpha=0.25, gamma=2.0, bce_weight=0.5):
    """
    Hybrid = BCE * bce_weight + FocalLoss * (1 - bce_weight)
    """
    def loss(y_true, y_pred):
        # --- Binary Cross Entropy ---
        bce = tf.keras.losses.binary_crossentropy(y_true, y_pred)

        # --- Focal Loss ---
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
        focal = -alpha * (1 - y_pred) ** gamma * y_true * K.log(y_pred) \
                - (1 - alpha) * y_pred ** gamma * (1 - y_true) * K.log(1 - y_pred)

        focal = K.mean(focal, axis=-1)

        # --- Combine Both ---
        return bce_weight * bce + (1 - bce_weight) * focal

    return loss

# ==================== CNN + LSTM MODEL ====================
def build_cnn_lstm(input_length):
    model = Sequential([
        # --- CNN Layers ---
        Conv1D(32, kernel_size=7, activation='relu', input_shape=(input_length, 1)),
        BatchNormalization(),
        MaxPooling1D(2),
        Dropout(0.2),

        Conv1D(64, kernel_size=5, activation='relu'),
        BatchNormalization(),
        MaxPooling1D(2),
        Dropout(0.3),

        Conv1D(128, kernel_size=3, activation='relu'),
        BatchNormalization(),
        MaxPooling1D(2),
        Dropout(0.4),
        
        # --- LSTM ---
        LSTM(64, return_sequences=False),

        # --- Dense Layers ---
        Dense(64, activation='relu'),
        Dropout(0.4),

        Dense(1, activation='sigmoid')
    ])

    model.compile(
        optimizer=Adam(1e-4),
        loss=hybrid_focal_loss(alpha=0.25, gamma=2.0, bce_weight=0.5),
        metrics=['accuracy']
    )
    return model

# ==================== DATA AUGMENTATION ====================
def augment_signal(signal):
    # 1) Very Light Noise
    noise = np.random.normal(0, 0.005, signal.shape)
    signal_noisy = signal + noise

    # 2) Small Time Shift
    shift = np.random.randint(-5, 5)
    signal_shifted = np.roll(signal_noisy, shift)

    # 3) Gentle Scaling
    scale = np.random.uniform(0.97, 1.03)
    signal_scaled = signal_shifted * scale

    return signal_scaled

def augment_batch(X, y):
    X_aug = []
    y_aug = []

    for i in range(len(X)):
        X_aug.append(X[i])
        y_aug.append(y[i])

        # Generate **1 weakly augmented version**
        X_aug.append(augment_signal(X[i]))
        y_aug.append(y[i])

    return np.array(X_aug), np.array(y_aug)

# ==================== APPLY DATA AUGMENTATION ====================
print("\n" + "="*70)
print("APPLYING DATA AUGMENTATION")
print("="*70)
print(f"Before augmentation: {X_train_final.shape}")

X_train_aug, y_train_aug = augment_batch(X_train_final, y_train_final)

print(f"After augmentation:  {X_train_aug.shape}")
print(f"Augmentation factor: 2x (1 original + 1 augmented per sample)")

# ==================== BUILD AND TRAIN MODEL ====================
os.makedirs("results(Interictal_VS_Normal_VS_Ictal)", exist_ok=True)

print("\n" + "="*70)
print("BUILDING MODEL")
print("="*70)

model = build_cnn_lstm(X_train_aug.shape[1])
model.summary()

# Handle class imbalance
cw = compute_class_weight('balanced', classes=np.unique(y_train_aug), y=y_train_aug)
class_weights = {0: cw[0], 1: cw[1]}
print(f"\nClass weights: {class_weights}")

# Callbacks
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=15,
    restore_best_weights=True,
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    min_lr=1e-7,
    verbose=1
)

callbacks = [early_stop, reduce_lr]

# Train model
print("\n" + "="*70)
print("TRAINING MODEL (No Data Leakage)")
print("="*70)

history = model.fit(
    X_train_aug, y_train_aug,
    epochs=40,
    batch_size=32,
    validation_data=(X_val, y_val),
    class_weight=class_weights,
    callbacks=callbacks,
    verbose=1
)

# ==================== EVALUATE ON TEST SET ====================
print("\n" + "="*70)
print("EVALUATING ON TEST SET")
print("="*70)

test_loss, test_acc = model.evaluate(X_test, y_test_encoded, verbose=0)
print(f"\n✅ Test Accuracy: {test_acc:.4f}")
print(f"✅ Test Loss:     {test_loss:.4f}")

# Save weights
weight_path = "results(Interictal_VS_Normal_VS_Ictal)/cnn_lstm_fixed.weights.h5"
model.save_weights(weight_path)
print(f"\n💾 Weights saved to {weight_path}")

# Save full model
model_path = "results(Interictal_VS_Normal_VS_Ictal)/cnn_lstm_fixed.keras"
model.save(model_path)
print(f"💾 Model saved to {model_path}")

# ==================== PREDICTIONS & METRICS ====================
y_pred_prob = model.predict(X_test, verbose=0)
y_pred = (y_pred_prob > 0.5).astype("int32").flatten()

print("\n" + "="*70)
print("CLASSIFICATION REPORT")
print("="*70)
print(classification_report(
    y_test_encoded, y_pred,
    target_names=['ICTAL', 'Others (Normal+Interictal)'],
    digits=4
))

# ==================== CONFUSION MATRIX ====================
cm = confusion_matrix(y_test_encoded, y_pred)

plt.figure(figsize=(8, 6))
sns.heatmap(
    cm, annot=True, fmt='d', cmap='Blues',
    xticklabels=['ICTAL', 'Others'],
    yticklabels=['ICTAL', 'Others'],
    annot_kws={'size': 14}
)
plt.title(f"Confusion Matrix (No Data Leakage)\nTest Accuracy: {test_acc:.4f}", fontsize=14)
plt.xlabel("Predicted", fontsize=12)
plt.ylabel("True", fontsize=12)
plt.tight_layout()
plt.savefig("results(Interictal_VS_Normal_VS_Ictal)/cnn_lstm_conf_fixed.png", dpi=300)
plt.show()

# Calculate detailed metrics
tn, fp, fn, tp = cm.ravel()

accuracy = (tp + tn) / np.sum(cm)
precision = tp / (tp + fp) if (tp + fp) > 0 else 0
recall = tp / (tp + fn) if (tp + fn) > 0 else 0
f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
specificity = tn / (tn + fp) if (tn + fp) > 0 else 0

print("\n" + "="*70)
print("DETAILED METRICS (No Data Leakage)")
print("="*70)
print(f"  Accuracy:    {accuracy:.4f}")
print(f"  Precision:   {precision:.4f}")
print(f"  Recall:      {recall:.4f}")
print(f"  F1-score:    {f1:.4f}")
print(f"  Specificity: {specificity:.4f}")

# ==================== TRAINING HISTORY ====================
plt.figure(figsize=(14, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy', linewidth=2)
plt.plot(history.history['val_accuracy'], label='Val Accuracy', linewidth=2)
plt.axhline(y=test_acc, color='r', linestyle='--', label=f'Test Acc: {test_acc:.4f}', linewidth=2)
plt.title('Model Accuracy (No Data Leakage)', fontsize=13)
plt.xlabel('Epoch', fontsize=11)
plt.ylabel('Accuracy', fontsize=11)
plt.legend(fontsize=10)
plt.grid(alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss', linewidth=2)
plt.plot(history.history['val_loss'], label='Val Loss', linewidth=2)
plt.title('Model Loss', fontsize=13)
plt.xlabel('Epoch', fontsize=11)
plt.ylabel('Loss', fontsize=11)
plt.legend(fontsize=10)
plt.grid(alpha=0.3)

plt.tight_layout()
plt.savefig("results(Interictal_VS_Normal_VS_Ictal)/cnn_lstm_history_fixed.png", dpi=300)
plt.show()

print("\n" + "="*70)
print("✅ CNN+LSTM TRAINING COMPLETED (No Data Leakage)!")
print("="*70)
print(f"\nKey improvements:")
print(f"  ✅ Files split BEFORE segmentation")
print(f"  ✅ No overlap between train/test files")
print(f"  ✅ Data augmentation applied only to training set")
print(f"  ✅ Validation set created from training split")
print(f"  ✅ Test set remains completely unseen")

LOADING PRE-SPLIT DATA (No Data Leakage)

Dataset shapes:
  Train:      (7246, 347, 1)
  Validation: (1554, 347, 1)
  Test:       (2200, 347, 1)

Class distribution:
  Train: (array([0, 1]), array([1449, 5797]))
  Val:   (array([0, 1]), array([ 311, 1243]))
  Test:  (array([0, 1]), array([ 440, 1760]))

APPLYING DATA AUGMENTATION
Before augmentation: (7246, 347, 1)
After augmentation:  (14492, 347, 1)
Augmentation factor: 2x (1 original + 1 augmented per sample)

BUILDING MODEL


c:\Users\Mostafa\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 341, 32)        │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 341, 32)        │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 170, 32)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 170, 32)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 166, 64)        │        10,304 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 166, 64)        │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 83, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 83, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 81, 128)        │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 81, 128)        │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_2 (MaxPooling1D)  │ (None, 40, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 40, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 89,793 (350.75 KB)

 Trainable params: 89,345 (349.00 KB)

 Non-trainable params: 448 (1.75 KB)


Class weights: {0: np.float64(2.500345065562457), 1: np.float64(0.6249784371226497)}

TRAINING MODEL (No Data Leakage)
Epoch 1/40
421/453 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step - accuracy: 0.3915 - loss: 0.3840